In [25]:
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
import pandas as pd
import os
from tqdm.notebook import tqdm

In [26]:
def convertToString(list):
    return ' '.join(list)
def process(input_path,output_path,file):
  df=pd.read_csv(f'{input_path}{file}')
  df['sentences'] = df['sentences'].apply(eval)
  df['sentences'] = df['sentences'].apply(lambda x:' '.join(x))
  corpus = df['sentences'].tolist()
  corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
  prior_cases_ids=[]
  prior_cases = []
  for i, row in tqdm(df.iterrows(),total=len(df)):
    query = df.loc[i]['sentences']
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings,)[0]
    top_results = torch.topk(cos_scores, k=4)
    document_ids=[]
    for score, idx in zip(top_results[0], top_results[1]):
      document_ids.append(idx.item())
    document_ids=document_ids[1:]
    
    prior_cases_ids.append(document_ids)
    prior_cases.append([corpus[i] for i in document_ids])
    
  df['prior_cases_ids']=prior_cases_ids
  df['prior_cases']=prior_cases
  df.to_csv(f"{output_path}{file}",index=False)

In [27]:
input_path= "./data/prior_case/groups/"
output_path= "./data/prior_case/results/"

files=os.listdir(input_path)
for file in files:
  process(input_path,output_path,file)

  0%|          | 0/354 [00:00<?, ?it/s]

  0%|          | 0/353 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/147 [00:00<?, ?it/s]